<a href="https://colab.research.google.com/github/matsu641/APS360Project/blob/main/APS360_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
# 1. Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. Data preprocessing
data_dir = "/content/drive/MyDrive/NIH_ChestXray_subset"

transform = transforms.Compose([
    transforms.Resize((128, 128)),      # Resize all images to 128x128
    transforms.ToTensor(),              # Convert to tensor
    transforms.Normalize((0.5,), (0.5,)) # Normalize to [-1, 1]
])

train_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
train_size = int(0.8 * len(train_dataset))
val_size   = len(train_dataset) - train_size
train_set, val_set = torch.utils.data.random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_set, batch_size=32, shuffle=False)


In [ ]:
# 3. Simple CNN architecture (baseline)
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=4):
        super(SimpleCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 32 * 32, 128), # After 2 poolings on 128x128
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

model = SimpleCNN(num_classes=4).to(device)

In [ ]:
# 4. Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# 5. Training loop
num_epochs = 5  # keep small for baseline
for epoch in range(num_epochs):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    train_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Loss: {running_loss/len(train_loader):.4f}, "
          f"Train Acc: {train_acc:.2f}%")

    # Validation
    model.eval()
    val_correct, val_total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            val_correct += predicted.eq(labels).sum().item()
            val_total += labels.size(0)
    val_acc = 100 * val_correct / val_total
    print(f"Validation Acc: {val_acc:.2f}%\n")


Epoch [1/5], Loss: 0.3418, Train Acc: 92.23%
Validation Acc: 95.49%

Epoch [2/5], Loss: 0.2976, Train Acc: 92.67%
Validation Acc: 95.49%

Epoch [3/5], Loss: 0.2893, Train Acc: 92.69%
Validation Acc: 95.49%

Epoch [4/5], Loss: 0.2693, Train Acc: 92.77%
Validation Acc: 95.49%

Epoch [5/5], Loss: 0.2450, Train Acc: 92.77%
Validation Acc: 95.49%



In [ ]:
# 6. Save baseline model
torch.save(model.state_dict(), "baseline_cnn.pth")
print("Model saved as baseline_cnn.pth")

Model saved as baseline_cnn.pth


In [ ]:
# 7. Evaluate the result properly
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Collect predictions and labels
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = outputs.max(1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))

# Precision, Recall, F1 per class
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=val_loader.dataset.dataset.classes))


Confusion Matrix:
[[   0    1   13    0]
 [   0    0   39    0]
 [   0    0 1229    0]
 [   0    0    5    0]]
Classification Report:
              precision    recall  f1-score   support

Cardiomegaly       0.00      0.00      0.00        14
    Effusion       0.00      0.00      0.00        39
  No Finding       0.96      1.00      0.98      1229
   Pneumonia       0.00      0.00      0.00         5

    accuracy                           0.95      1287
   macro avg       0.24      0.25      0.24      1287
weighted avg       0.91      0.95      0.93      1287



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
